In [ ]:
import pandas as pd
import numpy as np

import datetime as dt

In [ ]:
from IPython.core.display import display, HTML # Расширить рабочее поле ноутбука на весь экран
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pd.options.display.max_colwidth = 300 # Увеличить длинну отображаемых строк

# Генерация данных

In [ ]:
data = pd.DataFrame([
            ['16-10-2021', '23:56:10', 'A', 'a1', '0001'], 
            ['16-10-2021', '23:58:30', 'A', 'a1', '0002'],
            ['16-10-2021', '23:58:35', 'B', 'b1', '0003'],
            ['16-10-2021', '23:58:56', 'B', 'b2', '0004'],
            ['16-10-2021', '23:59:21', 'C', 'c1', '0005'],
            ['16-10-2021', '23:59:42', 'C', 'c1', '0006'],

            ['17-10-2021', '00:03:05', 'A', 'a1', '0007'],
            ['17-10-2021', '00:03:25', 'A', 'a2', '0008'],
            ['17-10-2021', '00:03:48', 'A', 'a3', '0009'],
            ['17-10-2021', '00:05:24', 'B', 'b1', '0010'],
            ['17-10-2021', '00:05:36', 'B', 'b1', '0011'],
            ['17-10-2021', '00:05:44', 'B', 'b2', '0012'],
            ['17-10-2021', '00:06:03', 'B', 'b2', '0013'],
            ['17-10-2021', '03:12:13', 'B', 'b3', '0014'],
            ['17-10-2021', '03:12:46', 'C', 'c1', '0015'],
            ['17-10-2021', '13:06:54', 'C', 'c1', '0016'], 
            ['17-10-2021', '13:12:10', 'C', 'c2', '0017'], 
            ['17-10-2021', '19:48:30', 'C', 'c2', '0018'], 

            ['18-10-2021', '00:06:05', 'C', 'c1', '0019'], 
            ['18-10-2021', '03:08:03', 'C', 'c2', '0020'],
            ['18-10-2021', '12:07:08', 'C', 'c3', '0021'],
    
            ['19-10-2021', '00:04:03', 'A', 'a1', '0022'], 
            ['19-10-2021', '02:08:20', 'A', 'a2', '0023'],
            ['19-10-2021', '10:03:08', 'B', 'a2', '0024'],
            ['19-10-2021', '12:07:08', 'B', 'b1', '0025'],
    
], columns=['str_date', 'str_time','group', 'sub_group','val_id'])

data['datetime'] = data.apply(lambda row:  dt.datetime.strptime(f'{row.str_date} {row.str_time}', '%d-%m-%Y %H:%M:%S'), axis=1)

data['date'] = data.datetime.dt.date
data['time'] = data.datetime.dt.time
data['week'] = data.datetime.dt.isocalendar().week # Получить номер недели
data['weekday'] = data.datetime.dt.strftime('%A') # Получить день недели (на английском)


datetime_cols = ['datetime', 'date','time','week','weekday']
group_cols = ['group', 'sub_group']
data = data[[*datetime_cols, *group_cols, 'val_id']]

data

# Pandas

## Таблица

In [ ]:
table = (
    data
        .pivot_table(
                    index=['date', 'weekday'], 
                    columns=['group','sub_group'], 
                    values='val_id', 
                    aggfunc='nunique'
        )
        .fillna(0)
)
table

In [ ]:
print(table.columns)
A_cols = [col for col in table.columns if 'A' in col]
B_cols = [col for col in table.columns if 'B' in col]
C_cols = [col for col in table.columns if 'C' in col]

In [ ]:
table_styled = (
    table.style

            .set_properties(**{
                'text-align': 'center', 
                'width': '200px'})
            .format({col: "{}" for col in table.columns})
            .background_gradient(subset=A_cols, cmap='Greens')
            .background_gradient(subset=B_cols, cmap='Greys')
            .background_gradient(subset=C_cols, cmap='Reds')
            .set_table_styles([{'selector': 'th', 'props': [('border-color', 'black'), 
                                                            ('border-style','solid')]}])
    
)

table_styled

# Plotly

https://plotly.com/python/

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

В Plotly есть два отдельных пакета для рисования графиков: express и graph_objects. Первый как, говорят создатели, новее быстрее и лучше. Однако во втором, на мой взгляд, удобнее проводить тонкую настройку графиков. Потому я чаще всего пользуюсь graph_objects  

## Bar chart - столбчатые диаграммы

### Быстрый вариант

In [ ]:
data.head()

In [ ]:
x_col = 'date'
y_col = 'val_id'
group_col = 'group'

df = data.groupby([x_col, group_col], as_index=False)[y_col].nunique()
print(df.to_markdown())

fig = px.bar(df, x=x_col, y=y_col, color=group_col, title="Динамика значений")
fig.show()

In [ ]:
x_col = 'date'
y_col = 'val_id'
group_col = 'group'

df = data.pivot_table(index=x_col, columns=group_col, values=y_col, aggfunc='nunique').fillna(0).reset_index()
print(df.to_markdown())

xaxis_title = "Дата"
yaxis_title = 'Количество значений'
title = 'Динамика значений'

fig = go.Figure(data=[
    go.Bar(name=group, x=df[x_col], y=df[group])
    
    for group in data[group_col].unique()
])

fig.update_layout(
    title = title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    width=800,
    height=600,
    barmode='stack',
)

fig.show()

### Диаграмма с текстами

In [ ]:
x_col = 'date'
y_col = 'val_id'
group_col = 'group'

df = data.pivot_table(index=x_col, columns=group_col, values=y_col, aggfunc='nunique').fillna(0).reset_index()

# Посчитаем проценты для каждой группы
groups = data[group_col].unique().tolist()
df['Sum'] = df[groups].sum(axis=1)
for group in data[group_col].unique():
    df[f"{group}%"] = df.apply(lambda row: f"{int(np.round(row[group]*100/row['Sum']))}%", axis=1)


print(df.to_markdown())

xaxis_title = "Дата"
yaxis_title = 'Количество значений'
title = 'Динамика значений'

fig = go.Figure(data=[
    go.Bar(
        name=group, 
        x=df[x_col], y=df[group], 
        text=df[f'{group}%'],  textposition = 'inside'
    )
    
    for group in data[group_col].unique()
])

fig.update_layout(
    title = title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    width=800,
    height=600,
    barmode='stack',
)

fig.show()

## Pie chart - круговые диаграммы

### Простой вариант с текстами

In [ ]:
df = data.group.value_counts().reset_index().rename(columns={'index': 'Группа','group': 'Количество'})
print(df.to_markdown())

fig = px.pie(df, names='Группа', values= 'Количество')
fig.update_traces(textposition='inside', textinfo='value+percent+label')

fig.show("notebook")

### Кргуговая диаграмма с текстами в легенде (бывает полезно, когда есть маленькие доли)

In [ ]:
df = data.group.value_counts().reset_index().rename(columns={'index': 'Группа','group': 'Количество'})

df['Процент'] = df.apply(lambda row: np.round(row['Количество']*100/df['Количество'].sum(), 3), axis=1)
df['Текст'] = df.apply(lambda row: f"{row['Группа']} ({row['Процент']}%) ({row['Количество']})", axis=1)

fig = go.Figure(data=[
    go.Pie(labels=df['Текст'], values=df['Процент'])
])




fig.update_traces(hoverinfo='label+percent', textinfo='none', textfont_size=20)
fig.show()

##  Line chart - линейный график

#### В некоторых случаях проще воспользоваться методом .scatter

In [ ]:
x_col = 'date'
y_col = 'val_id'
group_col = 'group'

df = data.pivot_table(index=x_col, columns=group_col, values=y_col, aggfunc='nunique').fillna(0).reset_index()
print(df.to_markdown())

xaxis_title = "Дата"
yaxis_title = 'Количество значений'
title = 'Динамика значений'

fig = go.Figure(data=[
    go.Scatter(x=df[x_col], y=df[group], name=group)
    
    for group in data[group_col].unique()

])

fig.update_layout(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title
)

fig.show()

### Скользящее среднее

In [ ]:
dates = pd.date_range(start='09/01/2021', end='10/30/2021')
values = np.random.choice(range(100), len(dates), replace=True)

df = pd.DataFrame({
    'date': dates,
    'value': values,
})

df['rolling'] = df.value.rolling(7).mean()

df.head(15)

### Вертикальные линии и выделения областей

- https://plotly.com/python/horizontal-vertical-shapes/

In [ ]:
x_col = 'date'
y_col1 = 'value'
y_col2 = 'rolling'

xaxis_title = "Дата"
yaxis_title = 'Количество'

fig = go.Figure(data=[
    go.Scatter(x=df[x_col], y=df[y_col1], marker_color='#D7DBDD', name = 'Количество в день'),
    go.Scatter(x=df[x_col], y=df[y_col2], marker_color='#4D5656', name = 'Среднее количество за неделю')
])

fig.update_layout(
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title
)

fig.add_vrect(
    x0='2021-09-16', 
    x1='2021-09-16',
    line_width=1, 
    line_dash="dash", 
    line_color="green",
    annotation_text="Началальная дата",

)


fig.add_vrect(
    x0='2021-10-10', 
    x1='2021-10-17', 
    fillcolor="green", 
    opacity=0.25, 
    line_width=0,
    annotation_text="Зеленая зона",

)

fig.add_hrect(
    y0=0, 
    y1=20,
    fillcolor="red",
    opacity=0.25, 
    line_width=0,
    annotation_text="Красная зона",

)

fig.show()

### Горизонтальные линии и разноцветные маркеры

## Визуализация катировок

In [ ]:
stock = pd.read_csv('data/stock.csv')
stock.head()

### Candlestick Charts - японские свечи
https://plotly.com/python/candlestick-charts/

In [ ]:
fig = go.Figure(data=[
    go.Candlestick(
        x=df['hour'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'])
])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

### OHLC Charts
https://plotly.com/python/ohlc-charts/

In [ ]:
fig = go.Figure(data=[
    go.Ohlc(
        x=df['hour'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'])
])

fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()